In [1]:
from udma import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cb=UDMA_CLASS("140.105.17.180", 7)
# cb=UDMA_CLASS("192.168.1.10", 7)
cb.connect()
cb.log(0)

[(5,), 0]

Comblock Registers

|Register | Connection |
| ----- | ---- |
|reg0_o | Config_Res_intan_A|
|reg1_o | Config_Res_intan_B|
|reg2_o | Config_Res_intan_C|
|reg3_o | Config_Res_intan_D|
|reg4_o | Read_intan(3:0)|
|reg5_o | Sys_reset (active low)|
|reg6_o | **Combintan_w_header**/nsamples |
|reg7_o | **Combintan_w_header**/sys_en |



In [3]:
def hicce_init():
    # Select comblock
    cb.select_comblock(0)
    # Reset HiCCE
    cb.write_reg(5, 0) #RESET ON
    cb.write_reg(5, 1) #RESET OFF

    #Disable Acquisiton 
    cb.write_reg(4, 0)
    cb.write_reg(7, 0)

    # FIFO Clear
    cb.select_comblock(0)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(1)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(0)

    # Configure INTANT (Reset Counter)
    cb.write_reg(0, 6150)
    cb.write_reg(1, 6150)
    cb.write_reg(2, 6150)
    cb.write_reg(3, 6150)
    # Configure INTANT (Cyclic Read)
    cb.write_reg(0, 6182)
    cb.write_reg(1, 6182)
    cb.write_reg(2, 6182)
    cb.write_reg(3, 6182)

def enable_acquisition(N=3200):
    #Set number of samples per package
    cb.write_reg(6, N)

    #Enable acquisition
    cb.write_reg(4, 15)
    cb.write_reg(7,1)

def read_channels(NCh=1, NSamp=3200):
    dataAB=[]
    dataCD=[]
    for i in range(NCh):
        cb.select_comblock(0)
        dataAB+=cb.read_fifo(NSamp+4)[1]
        cb.select_comblock(1)
        dataCD+=cb.read_fifo(NSamp+4)[1]
        return dataAB, dataCD


In [4]:
hicce_init()
enable_acquisition(320)


In [5]:
ab,cd=read_channels(1, 320)
print(ab)
print(cd)

[16777280, 1, 685907778, 4294238090, 4294954458, 4292811919, 4285406859, 4292038538, 4252458937, 4252112888, 4293867180, 4294917356, 4294257807, 4294903946, 4294903951, 4281736335, 4294062219, 4294917258, 4294909066, 4294921628, 4294954259, 4215725706, 4089659019, 4158326922, 4287043727, 4291697806, 4294909830, 4294123407, 4294905842, 4146727034, 4158724746, 4294449882, 4293876570, 4294931384, 4285948853, 4294955177, 4294955199, 4110620859, 4294646922, 4294921663, 4294925307, 4294907531, 4292296587, 4294905738, 4294912143, 4294904715, 4294904712, 4144110782, 4293865695, 4294917262, 4294909066, 4294921903, 4294954815, 4278640063, 4272503947, 4290841743, 4218821835, 4285669096, 4294912761, 4278323087, 4294904826, 4294905851, 4294908554, 4273676170, 4294398874, 4294927089, 4294927089, 4294909578, 4294909578, 4294952961, 4102670337, 4102623301, 996675653, 996678731, 4294909003, 4294905739, 4062318475, 4062334097, 4204219537, 4204205176, 4007597176, 4007597195, 4294907019, 4294914059, 42949

In [22]:
dpack=ab
head=dpack[0]
if head>>24 == 0x01:
    CEN=(head>>16) & 0xff
    SBT=(head>>8) & 0xff
    SAT=head & 0xff
    TS=dpack[1]<<32 | dpack[2]
    ch0=[]
    ch1=[]
    for d in dpack[3:]:
        if d>>24 ==0x04:
            CENT=(d>>24) & 0xff
            FLAGS=d & 0xffff
            break
        else:
            ch0.append[d>>16]
            ch1.append[d&0xffff]
    


532576002649